In [175]:
%pylab inline
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import pandas as pd

import devkit.api as dk
import finkit.api as fk
import DataAPI
import WindAPI.api as wind
import alpha

Populating the interactive namespace from numpy and matplotlib
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


---

# 数据库架构

In [1]:
db_names = [
    "index_contents",  # 股票列表
    "industry",  # 行业分类
    "indicator",  # 基础指标
    "factor",  # 因子
    "factor_return",  # 因子收益
]

# db_name = "INDICATOR"
for db_name in db_names:
    DataAPI.schema.show_db_info(db_name)

NameError: name 'DataAPI' is not defined

---

# 待更新日期

In [172]:
start = "2018-05-01"
end = "2018-08-31"
trading_days_ts = DataAPI.api.get_monthly_last_trading_days(start=start, end=end)
trading_days_fr = DataAPI.api.get_report_days(start=start, end=end)
# trading_days = DataAPI.api.get_monthly_last_trading_days(start=start, end=end)
trading_days = DataAPI.api.get_trading_days(start=start, end=end)
trading_days = sorted(trading_days)
# trading_days_ts = sorted(trading_days_ts)
# trading_days_fr = sorted(trading_days_fr)

In [123]:
# 确定股票池
stock_pool_zz500 = wind.get_index_contents(date=trading_days_ts[-1], index_code='000905.SH')['sec_id'].tolist()
stock_pool_hs300 = wind.get_index_contents(date=trading_days_ts[-1], index_code='000300.SH')['sec_id'].tolist()

In [173]:
# 更新zz500成分股及其指数收益率
zz500_stocks_rr_monthly = alpha.api.derive_next_month_rr(sec_ids=stock_pool_zz500, start_date='2005-01-01', end_date=end )
zz500_stocks_rr_monthly.to_csv(r"E:\07_data\02_factor\datasets\return_rate\zz500_stocks_rr_monthly.csv", index=0)

zz500_index_rr_monthly = alpha.api.derive_next_month_rr(sec_ids=['000905.SH'], start_date='2005-01-01', end_date=end )
zz500_index_rr_monthly.to_csv(r"E:\07_data\02_factor\datasets\return_rate\zz500_index_rr_monthly.csv", index=0)

# 更新hs300成分股及其指数收益率
# hs300_stocks_rr_monthly = alpha.api.derive_next_month_rr(sec_ids=stock_pool_hs300, start_date='2005-01-01', end_date=end )
# hs300_stocks_rr_monthly.to_csv(r"E:\07_data\02_factor\datasets\return_rate\hs300_stocks_rr_monthly.csv", index=0)

# hs300_index_rr_monthly = alpha.api.derive_next_month_rr(sec_ids=['000300.SH'], start_date='2005-01-01', end_date=end )
# hs300_index_rr_monthly.to_csv(r"E:\07_data\02_factor\datasets\return_rate\hs300_index_rr_monthly.csv", index=0)

In [30]:
# 确定更新指标
a = DataAPI.api.schema.get_schema('indicator')
b = pd.DataFrame(a).T
# b = b[b.aspect != '辅助指标']
feats_ts = b[b.type == '时间序列'].index.tolist()
feats_fr = b[b.type == '财报数据'].index.tolist()

---

# indicator

In [165]:
# DataAPI.write.update_indicators(indicators=['CLOSE'], sec_ids=['000300.SH', '000905.SH'], trading_days=trading_days, override=True, log=False)
DataAPI.write.update_indicators(indicators=['CLOSE'], sec_ids=stock_pool_zz500, trading_days=trading_days, override=True, log=False)

[INFO] 2018-09-27 00:23:25 schema updated: CLOSE


In [171]:
# indicators = inds_ts  # 为空表示更新所有存在的indicator
DataAPI.write.update_indicators(indicators=feats_ts, sec_ids=stock_pool_zz500, trading_days=trading_days_ts, override=True, log=False)

[INFO] 2018-09-27 00:56:01 schema updated: ADJFACTOR
[INFO] 2018-09-27 00:56:09 schema updated: AMT
[INFO] 2018-09-27 00:56:17 schema updated: BETA_100W
[INFO] 2018-09-27 00:56:19 schema updated: BIAS_120D
[INFO] 2018-09-27 00:56:21 schema updated: BIAS_20D
[INFO] 2018-09-27 00:56:22 schema updated: BIAS_60D
[INFO] 2018-09-27 00:56:23 schema updated: CLOSE
[INFO] 2018-09-27 00:56:28 schema updated: DIVIDENDYIELD2
[INFO] 2018-09-27 00:56:29 schema updated: DOWN_DAYS
[INFO] 2018-09-27 00:56:32 schema updated: EV2_TO_EBITDA
[INFO] 2018-09-27 00:56:34 schema updated: HIGH
[INFO] 2018-09-27 00:56:35 schema updated: IPO_LISTDAYS
[INFO] 2018-09-27 00:56:36 schema updated: LOW
[INFO] 2018-09-27 00:56:44 schema updated: MKT_CAP_FLOAT
[INFO] 2018-09-27 00:56:54 schema updated: PB_LF
[INFO] 2018-09-27 00:57:04 schema updated: PCF_OCF_TTM
[INFO] 2018-09-27 00:57:15 schema updated: PS_TTM
[INFO] 2018-09-27 00:57:16 schema updated: RISK_GAINVARIANCE120
[INFO] 2018-09-27 00:57:17 schema updated: RISK

In [120]:
# indicators = inds_ts  # 为空表示更新所有存在的indicator
# 加了sec_ids后 慎用override 会删除数据
DataAPI.write.update_indicators(indicators=feats_fr, sec_ids=stock_pool_zz500, trading_days=trading_days_fr, override=True, log=False)

IndexError: list index out of range

---

# factor

In [28]:
a = pd.DataFrame(DataAPI.schema.get_schema("factor")).T
factors = a.sort_values(['level']).index.tolist()

In [30]:
factors

['DIVIDENDYIELD2',
 'EV2_TO_EBITDA',
 'PB_LF',
 'PCF_NCF_TTM',
 'PS_TTM',
 'VAL_PE_DEDUCTED_TTM']

In [35]:
# a = pd.DataFrame(DataAPI.schema.get_schema("indicator")).T
# inds_ts = a[a.type == "时间序列"].index.tolist()
# factors4 = ["LEVERAGE","PROFIT"] # 基于其他做好的factor
# DataAPI.write.update_factors(factors=factors1,trading_days=trading_days , override=False, log=False)
# DataAPI.write.update_factors(factors=factors2,trading_days=trading_days , override=False, log=False)
# DataAPI.write.update_factors(factors=factors3,trading_days=trading_days , override=False, log=False)
DataAPI.write.update_factors(factors=factors,trading_days=trading_days , override=False, log=False)

[INFO] 2018-06-28 14:14:56 Updating factor DIVIDENDYIELD2
[INFO] 2018-06-28 14:14:56 schema updated: DIVIDENDYIELD2
[INFO] 2018-06-28 14:14:56 Updating factor EV2_TO_EBITDA
[INFO] 2018-06-28 14:14:56 schema updated: EV2_TO_EBITDA
[INFO] 2018-06-28 14:14:56 Updating factor PB_LF
[INFO] 2018-06-28 14:14:56 schema updated: PB_LF
[ERROR] 2018-06-28 14:14:57 Unrecognized factor: PCF_NCF_TTM
[INFO] 2018-06-28 14:14:57 ------------------------------------------
[INFO] 2018-06-28 14:14:57 Updating factor PS_TTM
[INFO] 2018-06-28 14:14:57 TABLE PS_TTM is created in E:\02_work\sw\db\factor\2005.db.
[INFO] 2018-06-28 14:14:57 INDEX PS_TTM_index is created on TABLE PS_TTM of E:\02_work\sw\db\factor\2005.db.
[INFO] 2018-06-28 14:14:59 TABLE PS_TTM is created in E:\02_work\sw\db\factor\2006.db.
[INFO] 2018-06-28 14:14:59 INDEX PS_TTM_index is created on TABLE PS_TTM of E:\02_work\sw\db\factor\2006.db.
[INFO] 2018-06-28 14:15:02 TABLE PS_TTM is created in E:\02_work\sw\db\factor\2007.db.
[INFO] 2018-

In [5]:
DataAPI.write.update_factors(factors=['GROWTH'],trading_days=trading_days, override=True, log=False)

[ERROR] 2018-06-28 13:18:32 Unrecognized factor: GROWTH
[INFO] 2018-06-28 13:18:32 ------------------------------------------


---

# factor_return

In [27]:
# factors = ["SIZE", 'VALUE', 'BETA', 'VOL', 'ROE', 'MOM', 'LIQ'] # 为空表示更新所有存在的factors
factors = []
DataAPI.write.update_factors_return(['GROWTH'], trading_days)

[INFO] 2018-03-14 16:13:50 Updating factor_return GROWTH...
[INFO] 2018-03-14 16:14:57 存量更新：更新当前已经有的记录
[INFO] 2018-03-14 16:14:57 schema updated: GROWTH
[INFO] 2018-03-14 16:14:57 传入日期数：1015  已经存在个数：1  实际写入次数：1014
[INFO] 2018-03-14 16:14:57 factor_return GROWTH is updated.
[INFO] 2018-03-14 16:14:57 ------------------------------------------
